# 1. Берём 2 датасета


In [22]:
# Загружаем датасет 'ag_news' (определяем сообщение - спам/не спам)
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("ag_news")

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
# всего 4 класса, по 25% каждого в dataset['train']
# сформирую датасет из 8000 примеров, по 2000 примеров на каждый класс

count0, count1, count2, count3 = 0, 0, 0, 0
dataset_short = []
for i in range(len(dataset['train'])):
  if dataset['train'][i]['label'] == 0 and count0 < 2000:
    dataset_short.append({'news': dataset['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset['train'][i]['label'] == 1 and count1 < 2000:
    dataset_short.append({'news': dataset['train'][i]['text'], 'label': 1})
    count1 += 1
  elif dataset['train'][i]['label'] == 2 and count2 < 2000:
    dataset_short.append({'news': dataset['train'][i]['text'], 'label': 2})
    count2 += 1
  elif dataset['train'][i]['label'] == 3 and count3 < 2000:
    dataset_short.append({'news': dataset['train'][i]['text'], 'label': 3})
    count3 += 1
len(dataset_short)

In [ ]:
import random
random.shuffle(dataset_short)
dataset = {'train': dataset_short[:6400], 'test': dataset_short[6400:]}

In [ ]:
# Выводим объём выборок
print('Объём train:', len(dataset['train']))
print('Объём test:', len(dataset['test']))

train_texts_news = [s['news'] for s in dataset['train']]
test_texts_news = [s['news'] for s in dataset['test']]

train_labels_news = [s['label'] for s in dataset['train']]
test_labels_news = [s['label'] for s in dataset['test']]

# ВЫводим примеры
print(train_texts_news[0], train_labels_news[0])
print(test_texts_news[0], test_labels_news[0])

In [ ]:
dataset_imdb = load_dataset("imdb")

count0, count1 = 0, 0
dataset_short_imdb = []

for i in range(len(dataset_imdb['train'])):
  if dataset_imdb['train'][i]['label'] == 0 and count0 < 4000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_imdb['train'][i]['label'] == 1 and count1 < 4000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 1})
    count1 += 1
len(dataset_short_imdb)

In [ ]:
random.shuffle(dataset_short_imdb)
dataset_imdb = {'train': dataset_short_imdb[:6400], 'test': dataset_short_imdb[6400:]}

In [ ]:
# Выводим объём выборок
print('Объём train:', len(dataset_imdb['train']))
print('Объём test:', len(dataset_imdb['test']))

train_texts_imdb = [s['text'] for s in dataset_imdb['train']]
test_texts_imdb = [s['text'] for s in dataset_imdb['test']]

train_labels_imdb = [s['label'] for s in dataset_imdb['train']]
test_labels_imdb = [s['label'] for s in dataset_imdb['test']]

In [ ]:
print(train_texts_imdb[0])
print(train_labels_imdb[0])
print()
print(test_texts_imdb[0])
print(test_labels_imdb[0])

# 2. Прописываем функции предобработки текста

In [ ]:
# news example
news_example = train_texts_news[0]
print(news_example)

# imdb example
imdb_example = train_texts_imdb[0]
print(imdb_example)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(set(stopwords.words("english")))

def stopwords_special(text):
  text = text.lower()
  tokens = word_tokenize(text)
  tokens = [word for word in tokens if word not in stop_words]  # убираем стоп-слова
  tokens = word_tokenize(re.sub(r'[^a-zA-Zа-яА-Я ]', '', ' '.join(tokens)))  # убираем спец символы, числа и знаки препинания
  return tokens

In [ ]:
print(stopwords_special(news_example))
print(stopwords_special(imdb_example))

In [ ]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger_eng')

def nouns_adj(text):
  tokens = stopwords_special(text)
  tagged_token = pos_tag(tokens)
  res = [word for word, label in tagged_token if label.startswith('N') or label.startswith('J')]  # береём только ИС и ИП
  return res

In [ ]:
print(nouns_adj(news_example))
print(nouns_adj(imdb_example))

In [ ]:
# стемминг с удалением стоп-слов, спец.символов, знаков препинания + оставляем ИС и ИП
nltk.download('wordnet')
nltk.download('omw-1.4')

def stemming(text):
  tokens = nouns_adj(text)

  stemmer = nltk.PorterStemmer()  # инициализируем стеммер
  stemmed_tokens = [stemmer.stem(token) for token in tokens]  # перебираем токены и применяем алгоритм стемминга

  return stemmed_tokens

In [ ]:
print(stemming(news_example))
print(stemming(imdb_example))

In [ ]:
def lemma(text):
  tokens = nouns_adj(text)
  lemmatizer = nltk.WordNetLemmatizer()
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

  return lemmatized_tokens

In [ ]:
print(lemma(news_example))
print(lemma(imdb_example))

# 3. Универсальная функция векторизации (через CountVectorizer, TfidfVectorizer)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# method - "count"/"tfdf"
# edit - 1 (stopwords_special), 2 (nouns_adj), 3 (стемминг), 4 (лемматизация)

def vectorize(train_texts, test_texts, method, edit):
  if method == "count":
    vectorizer = CountVectorizer()
  else:
    vectorizer = TfidfVectorizer()

  if edit == 1:
    train_texts = [' '.join(stopwords_special(text)) for text in train_texts]
    test_texts = [' '.join(stopwords_special(text)) for text in test_texts]
  elif edit == 2:
    train_texts = [' '.join(nouns_adj(text)) for text in train_texts]
    test_texts = [' '.join(nouns_adj(text)) for text in test_texts]
  elif edit == 3:
    train_texts = [' '.join(stemming(text)) for text in train_texts]
    test_texts = [' '.join(stemming(text)) for text in test_texts]
  else:
    train_texts = [' '.join(lemma(text)) for text in train_texts]
    test_texts = [' '.join(lemma(text)) for text in test_texts]

  train_text = vectorizer.fit_transform(train_texts)
  test_text = vectorizer.transform(test_texts)

  return train_text, test_text, vectorizer

In [ ]:
for index in ['n', 'i']:
  for m in ['count', 'tfidf']:
    for edit in range(1, 5):
      res = 'Dataset: '
      if index == 'n':
        res += 'news, Vectorizer: '
        train, test = train_texts_news, test_texts_news
      else:
        res += 'imdb, Vectorizer: '
        train, test = train_texts_imdb, test_texts_imdb

      if m == 'count':
        res += 'count, Edit method: '
      else:
        res += 'tfidf, Edit method: '

      if edit == 1:
        res += 'stopwords_special'
      elif edit == 2:
        res += 'nouns_adj'
      elif edit == 3:
        res += 'stemming'
      else:
        res += 'lemma'

      train_text, test_text = vectorize(train, test, m, edit)[0:2]
      print(res)
      print('Объем train: ', len(train_text.toarray()), 'Объём test: ', len(test_text.toarray()))
      print('Размерность Train: ', len(train_text.toarray()[0]), 'Размерность Test: ', len(test_text.toarray()[0]))
      print()


In [21]:
# импортируем DecisionTree и метрики
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import pandas as pd

# на каждом из предыдущих вариантов обучаем - выводим таблицу метрик и там, где vectorizer - матрицу признаков tf-idf
for index in ['n', 'i']:
  for m in ['count', 'tfidf']:
    for edit in range(1, 5):
      res = 'Dataset: '
      if index == 'n':
        res += 'news, Vectorizer: '
        train_text, test_text, train_label, test_label = train_texts_news, test_texts_news, train_labels_news, test_labels_news
      else:
        res += 'imdb, Vectorizer: '
        train_text, test_text, train_label, test_label = train_texts_imdb, test_texts_imdb, train_labels_imdb, test_labels_imdb

      if m == 'count':
        res += 'count, Edit method: '
      else:
        res += 'tfidf, Edit method: '

      if edit == 1:
        res += 'stopwords_special'
      elif edit == 2:
        res += 'nouns_adj'
      elif edit == 3:
        res += 'stemming'
      else:
        res += 'lemma'

      vect = vectorize(train_text, test_text, m, edit)
      train_text, test_text = vect[0:2]

      # Обучение модели на тренировочных данных
      dtc = DecisionTreeClassifier(random_state=42)
      dtc.fit(train_text, train_label)

      # Предсказание на тестовых данных
      y_pred = dtc.predict(test_text)
      # Выводим F1-score по разным методам усреднения
      report = classification_report(test_label, y_pred, digits=3)
      print(res)
      print(report)

      if m == 'tfidf':
        if index == 'n':
          # Преобразуем матрицу в DataFrame для удобства
          df = pd.DataFrame(vect[2].fit_transform(train_texts_news).toarray(), columns=vect[2].get_feature_names_out())
          print(df)
        else:
          # Преобразуем матрицу в DataFrame для удобства
          df = pd.DataFrame(vect[2].fit_transform(train_texts_imdb).toarray(), columns=vect[2].get_feature_names_out())
          print(df)

Dataset: news, Vectorizer: count, Edit method: stopwords_special
              precision    recall  f1-score   support

           0      0.758     0.746     0.752       390
           1      0.779     0.871     0.823       410
           2      0.721     0.698     0.709       388
           3      0.715     0.663     0.688       412

    accuracy                          0.745      1600
   macro avg      0.743     0.744     0.743      1600
weighted avg      0.743     0.745     0.743      1600

Dataset: news, Vectorizer: count, Edit method: nouns_adj
              precision    recall  f1-score   support

           0      0.754     0.700     0.726       390
           1      0.757     0.866     0.808       410
           2      0.736     0.698     0.717       388
           3      0.733     0.714     0.723       412

    accuracy                          0.746      1600
   macro avg      0.745     0.744     0.743      1600
weighted avg      0.745     0.746     0.744      1600

Dataset:

In [23]:
# обучаем random_forest
from sklearn.ensemble import RandomForestClassifier

# на каждом из предыдущих вариантов обучаем - выводим таблицу метрик и там, где vectorizer - матрицу признаков tf-idf
for index in ['n', 'i']:
  for m in ['count', 'tfidf']:
    for edit in range(1, 5):
      res = 'Dataset: '
      if index == 'n':
        res += 'news, Vectorizer: '
        train_text, test_text, train_label, test_label = train_texts_news, test_texts_news, train_labels_news, test_labels_news
        rf = RandomForestClassifier(n_estimators=200, random_state=42)
      else:
        res += 'imdb, Vectorizer: '
        train_text, test_text, train_label, test_label = train_texts_imdb, test_texts_imdb, train_labels_imdb, test_labels_imdb
        rf = RandomForestClassifier(n_estimators=300, random_state=42)

      if m == 'count':
        res += 'count, Edit method: '
      else:
        res += 'tfidf, Edit method: '

      if edit == 1:
        res += 'stopwords_special'
      elif edit == 2:
        res += 'nouns_adj'
      elif edit == 3:
        res += 'stemming'
      else:
        res += 'lemma'

      vect = vectorize(train_text, test_text, m, edit)
      train_text, test_text = vect[0:2]

      # Обучение модели на тренировочных данных
      rf.fit(train_text, train_label)

      # Предсказание на тестовых данных
      y_pred = rf.predict(test_text)
      # Выводим F1-score по разным методам усреднения
      report = classification_report(test_label, y_pred, digits=3)
      print(res)
      print(report)

      if m == 'tfidf':
        if index == 's':
          # Преобразуем матрицу в DataFrame для удобства
          df = pd.DataFrame(vect[2].fit_transform(train_texts_news).toarray(), columns=vect[2].get_feature_names_out())
          print(df)
        else:
          # Преобразуем матрицу в DataFrame для удобства
          df = pd.DataFrame(vect[2].fit_transform(train_texts_imdb).toarray(), columns=vect[2].get_feature_names_out())
          print(df)


Dataset: news, Vectorizer: count, Edit method: stopwords_special
              precision    recall  f1-score   support

           0      0.894     0.803     0.846       390
           1      0.858     0.956     0.904       410
           2      0.822     0.786     0.804       388
           3      0.775     0.794     0.784       412

    accuracy                          0.836      1600
   macro avg      0.837     0.835     0.835      1600
weighted avg      0.837     0.836     0.835      1600

Dataset: news, Vectorizer: count, Edit method: nouns_adj
              precision    recall  f1-score   support

           0      0.880     0.790     0.832       390
           1      0.844     0.949     0.893       410
           2      0.827     0.776     0.801       388
           3      0.774     0.799     0.786       412

    accuracy                          0.829      1600
   macro avg      0.831     0.828     0.828      1600
weighted avg      0.831     0.829     0.828      1600

Dataset:

In [24]:
# теперь обучаем на GBM
from sklearn.ensemble import GradientBoostingClassifier

# на каждом из предыдущих вариантов обучаем - выводим таблицу метрик и там, где vectorizer - матрицу признаков tf-idf
for index in ['n', 'i']:
  for m in ['count', 'tfidf']:
    for edit in range(1, 5):
      res = 'Dataset: '
      if index == 'n':
        res += 'news, Vectorizer: '
        train_text, test_text, train_label, test_label = train_texts_news, test_texts_news, train_labels_news, test_labels_news
        gbm = GradientBoostingClassifier(
            n_estimators=300,        # Количество деревьев
            learning_rate=0.05,      # Скорость обучения
            max_depth=4,             # Глубина деревьев
            min_samples_split=10,    # Минимальное количество объектов для разбиения
            min_samples_leaf=5,      # Минимальный размер листа
            random_state=42)
      else:
        res += 'imdb, Vectorizer: '
        train_text, test_text, train_label, test_label = train_texts_imdb, test_texts_imdb, train_labels_imdb, test_labels_imdb
        gbm = GradientBoostingClassifier(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=4,
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=42)

      if m == 'count':
        res += 'count, Edit method: '
      else:
        res += 'tfidf, Edit method: '

      if edit == 1:
        res += 'stopwords_special'
      elif edit == 2:
        res += 'nouns_adj'
      elif edit == 3:
        res += 'stemming'
      else:
        res += 'lemma'

      vect = vectorize(train_text, test_text, m, edit)
      train_text, test_text = vect[0:2]

      # Обучение модели на тренировочных данных
      gbm.fit(train_text, train_label)

      # Предсказание на тестовых данных
      y_pred = gbm.predict(test_text)
      # Выводим F1-score по разным методам усреднения
      report = classification_report(test_label, y_pred, digits=3)
      print(res)
      print(report)

      if m == 'tfidf':
        if index == 's':
          # Преобразуем матрицу в DataFrame для удобства
          df = pd.DataFrame(vect[2].fit_transform(train_texts_news).toarray(), columns=vect[2].get_feature_names_out())
          print(df)
        else:
          # Преобразуем матрицу в DataFrame для удобства
          df = pd.DataFrame(vect[2].fit_transform(train_texts_imdb).toarray(), columns=vect[2].get_feature_names_out())
          print(df)

Dataset: news, Vectorizer: count, Edit method: stopwords_special
              precision    recall  f1-score   support

           0      0.864     0.813     0.838       390
           1      0.907     0.949     0.927       410
           2      0.807     0.817     0.812       388
           3      0.803     0.801     0.802       412

    accuracy                          0.846      1600
   macro avg      0.845     0.845     0.845      1600
weighted avg      0.845     0.846     0.845      1600

Dataset: news, Vectorizer: count, Edit method: nouns_adj
              precision    recall  f1-score   support

           0      0.846     0.803     0.824       390
           1      0.893     0.937     0.914       410
           2      0.829     0.789     0.808       388
           3      0.782     0.818     0.800       412

    accuracy                          0.838      1600
   macro avg      0.838     0.836     0.836      1600
weighted avg      0.837     0.838     0.837      1600

Dataset:

In [30]:
# обучаем AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# на каждом из предыдущих вариантов обучаем - выводим таблицу метрик и там, где vectorizer - матрицу признаков tf-idf
for index in ['n', 'i']:
  for m in ['count', 'tfidf']:
    for edit in range(1, 5):
      res = 'Dataset: '
      if index == 'n':
        res += 'news, Vectorizer: '
        train_text, test_text, train_label, test_label = train_texts_news, test_texts_news, train_labels_news, test_labels_news
        adaboost = AdaBoostClassifier(
          estimator=DecisionTreeClassifier(max_depth=2),  # Базовое дерево
          n_estimators=200,      # Больше деревьев для точности
          learning_rate=0.1,     # Оптимальный баланс между скоростью и качеством
          random_state=42)
      else:
        res += 'imdb, Vectorizer: '
        train_text, test_text, train_label, test_label = train_texts_imdb, test_texts_imdb, train_labels_imdb, test_labels_imdb
        adaboost = AdaBoostClassifier(
          estimator=DecisionTreeClassifier(max_depth=2),  # Базовое дерево
          n_estimators=200,      # Больше деревьев для точности
          learning_rate=0.1,     # Оптимальный баланс между скоростью и качеством
          random_state=42)

      if m == 'count':
        res += 'count, Edit method: '
      else:
        res += 'tfidf, Edit method: '

      if edit == 1:
        res += 'stopwords_special'
      elif edit == 2:
        res += 'nouns_adj'
      elif edit == 3:
        res += 'stemming'
      else:
        res += 'lemma'

      vect = vectorize(train_text, test_text, m, edit)
      train_text, test_text = vect[0:2]

      # Обучение модели на тренировочных данных
      adaboost.fit(train_text, train_label)

      # Предсказание на тестовых данных
      y_pred = adaboost.predict(test_text)
      # Выводим F1-score по разным методам усреднения
      report = classification_report(test_label, y_pred, digits=3)
      print(res)
      print(report)

      if m == 'tfidf':
        if index == 's':
          # Преобразуем матрицу в DataFrame для удобства
          df = pd.DataFrame(vect[2].fit_transform(train_texts_news).toarray(), columns=vect[2].get_feature_names_out())
          print(df)
        else:
          # Преобразуем матрицу в DataFrame для удобства
          df = pd.DataFrame(vect[2].fit_transform(train_texts_imdb).toarray(), columns=vect[2].get_feature_names_out())
          print(df)

Dataset: news, Vectorizer: count, Edit method: stopwords_special
              precision    recall  f1-score   support

           0      0.655     0.467     0.545       390
           1      0.858     0.573     0.687       410
           2      0.672     0.448     0.538       388
           3      0.399     0.765     0.525       412

    accuracy                          0.566      1600
   macro avg      0.646     0.563     0.574      1600
weighted avg      0.645     0.566     0.574      1600

Dataset: news, Vectorizer: count, Edit method: nouns_adj
              precision    recall  f1-score   support

           0      0.639     0.485     0.551       390
           1      0.847     0.539     0.659       410
           2      0.707     0.405     0.515       388
           3      0.404     0.806     0.539       412

    accuracy                          0.562      1600
   macro avg      0.649     0.559     0.566      1600
weighted avg      0.648     0.562     0.567      1600

Dataset: